# Regresion logistica
En una regresión logística se tiene una variable objetivo $Y$ la cual es binaria y es de interes para el investigador.
Se quiere construir una función $f(X)$ donde $X=(X_1, \ldots, X_p)$ es un conjunto de variables exogenas que se utilizaran para pronosticar a $Y$.

En un modelo de regresión lineal logística, se usan las funciones del tipo:
$$\ln \left(\frac{p}{1-p} \right)=\beta_0 +\beta_1X_1+\beta_2X_2+...+\beta_pX_p  $$ 
donde $p=P(Y=1)$

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, classification_report
from sklearn.feature_selection import RFE

In [ ]:
url="https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/Cancer.csv"
cancer=pd.read_csv(url, sep=";", index_col=0)
Y=cancer["Tipo"]
X=cancer.drop("Tipo", axis=1)
X_train, X_test, y_train, y_test=train_test_split(X, Y, test_size=0.2, stratify=Y)
X_train.shape

In [ ]:
logreg=LogisticRegression()
logreg.fit(X_train, y_train)
pd.DataFrame(np.transpose(logreg.coef_), index=X_train.columns)

In [ ]:
np.round(logreg.predict_proba(X_train),3)## Probabilidades

In [ ]:
logreg.predict(X_train)[0:10]

In [ ]:
y_pred=logreg.predict(X_train)
print(classification_report(y_train, y_pred))

# Curva ROC

Sirve para escoger el umbral de decisión

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.preprocessing import label_binarize
Y=cancer["Tipo"] ## Selecciono a Y
X=cancer.drop("Tipo", axis=1) ## Selecciono las X
Y = label_binarize(Y, classes=["B", "M"]) ## Vuelvo a Y de 0 y 1 (dummy)
X_train, X_test, y_train, y_test=train_test_split(X, Y, test_size=0.2, stratify=Y) ## Divido en entrenamiento y prueba
logreg=LogisticRegression() ### Llamo la regresion
logreg.fit(X_train, y_train)  ### La ajusto en entrenamiento
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))  ## Calculo el area bajo la curva
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1]) ## Calculo la fpr(false positive rate),
## la tpr(true positive rate) y thresholds (los umbrales)
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

In [ ]:
paraumbral=pd.DataFrame(np.transpose(np.round(roc_curve(y_test, logreg.predict_proba(X_test)[:,1]),4)), columns=["fpr", "tpr", "umbral"])
paraumbral

In [ ]:
thres=0.5761
y_pred= np.where(logreg.predict_proba(X_train)[:,1] > thres, 1, 0)
y_pred

In [ ]:
thres=0.5
y_pred= np.where(logreg.predict_proba(X_train)[:,1] > thres, 1, 0)
print(classification_report(y_train, y_pred)) ### Esto con el umbral de 0.5761

In [ ]:
thres=0.5 ## Conjunto de datos de prueba
y_pred= np.where(logreg.predict_proba(X_test)[:,1] > thres, 1, 0)
print(classification_report(y_test, y_pred))

# Para selección automática de caracteristicas

In [ ]:
logreg1=LogisticRegression() ### LLamo a la regresión logistica
rfe=RFE(logreg1, n_features_to_select=10) ## Le digo que solo me seleccione 10 variables mas importantes
## Pueden ser menos si lo desea.
rfe.fit(X_train, y_train)
X_train.columns[rfe.ranking_==1]

## Despues de seleccionar las 10 variables, ajusto el modelo

In [ ]:
Xnueva_train=X_train[X_train.columns[rfe.ranking_==1]] ## Selecciono de X las variables más importantes
Xnueva_test=X_test[X_test.columns[rfe.ranking_==1]] ## Hago lo mismo en el conjunto de datos de prueba
logreg.fit(Xnueva_train, y_train) ## Ajusto el modelo
y_pred=logreg.predict(Xnueva_train) ## Pronostico el entrenamiento
print(classification_report(y_train, y_pred)) ### Evaluo en el entrenamiento

## AUC con solo 10 variables

In [ ]:
logreg=LogisticRegression() ### Llamo la regresion
logreg.fit(Xnueva_train, y_train)  ### La ajusto en entrenamiento
logit_roc_auc = roc_auc_score(y_test, logreg.predict(Xnueva_test))  ## Calculo el area bajo la curva
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(Xnueva_test)[:,1]) ## Calculo la fpr(false positive rate),
# la tpr(true positive rate) y thresholds (los umbrales)
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

In [ ]:
paraumbral=pd.DataFrame(np.transpose(np.round(roc_curve(y_test, logreg.predict_proba(Xnueva_test)[:,1]),4)), columns=["fpr", "tpr", "umbral"])
paraumbral ## Selecciono el umbral

In [ ]:
thres=0.5172 ## Conjunto de datos de prueba
y_pred= np.where(logreg.predict_proba(Xnueva_test)[:,1] > thres, 1, 0)
print(classification_report(y_test, y_pred))

# Spam

In [ ]:
url="https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/spam.csv"
email=pd.read_csv(url, sep=";", decimal=",")
Y=email["spam"]
X=email.drop(["spam", "time"], axis=1)
X=pd.get_dummies(X, drop_first=True) 
X_train, X_test, y_train, y_test=train_test_split(X, Y, test_size=0.2, stratify=Y)
X_train.shape

In [ ]:
logreg=LogisticRegression()
logreg.fit(X_train, y_train)
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

In [ ]:
paraumbral=pd.DataFrame(np.transpose(np.round(roc_curve(y_test, logreg.predict_proba(X_test)[:,1]),4)), columns=["fpr", "tpr", "umbral"])
paraumbral[paraumbral["tpr"]>0.8]

In [ ]:
thres=0.1105
y_pred= np.where(logreg.predict_proba(X_test)[:,1] > thres, 1, 0)
print(classification_report(y_test, y_pred))

# Tarea

Con la base [Diabetes](https://github.com/Cruzalirio/Ucentral/blob/master/Bases/Diabetes.csv) realizar una árbol y una regresión logística y compararlos